# मजकूर वर्गीकरण कार्य

जसे आपण आधी उल्लेख केले आहे, आपण **AG_NEWS** डेटासेटवर आधारित एक साधे मजकूर वर्गीकरण कार्यावर लक्ष केंद्रित करू, ज्यामध्ये बातम्यांच्या मथळ्यांना चार श्रेणींमध्ये वर्गीकृत करायचे आहे: जागतिक, क्रीडा, व्यवसाय आणि विज्ञान/तंत्रज्ञान.

## डेटासेट

हा डेटासेट [`torchtext`](https://github.com/pytorch/text) मॉड्यूलमध्ये अंगभूत आहे, त्यामुळे आपण याला सहजपणे प्रवेश करू शकतो.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

येथे, `train_dataset` आणि `test_dataset` मध्ये अशा संग्रहांचा समावेश आहे जे अनुक्रमे वर्गाचा क्रमांक (लेबल) आणि मजकूराची जोड परत करतात, उदाहरणार्थ:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

मग, चला आपल्या डेटासेटमधील पहिल्या 10 नवीन हेडलाईन्स प्रिंट करूया:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

डेटासेट्स इटरेटर असल्यामुळे, जर आपल्याला डेटा अनेक वेळा वापरायचा असेल तर आपल्याला तो यादीमध्ये रूपांतरित करणे आवश्यक आहे:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## टोकनायझेशन

आता आपल्याला मजकूर **संख्यांमध्ये** रूपांतरित करायचा आहे ज्याचे प्रतिनिधित्व टेन्सर्स म्हणून करता येईल. जर आपल्याला शब्द-स्तरीय प्रतिनिधित्व हवे असेल, तर आपल्याला दोन गोष्टी कराव्या लागतील:
* **टोकनायझर** वापरून मजकूर **टोकन्स** मध्ये विभाजित करणे
* त्या टोकन्सचे **शब्दसंग्रह** तयार करणे.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

शब्दसंग्रह वापरून, आपण सहजपणे आमच्या टोकन केलेल्या स्ट्रिंगला संख्यांच्या संचामध्ये एन्कोड करू शकतो:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## शब्दांची पिशवी (Bag of Words) मजकूराचे प्रतिनिधित्व

कारण शब्द अर्थ व्यक्त करतात, कधी कधी आपण फक्त स्वतंत्र शब्दांकडे पाहून, वाक्यातील त्यांच्या क्रमाकडे दुर्लक्ष करून, मजकूराचा अर्थ समजू शकतो. उदाहरणार्थ, जेव्हा बातम्या वर्गीकृत करायच्या असतात, तेव्हा *weather* (हवामान), *snow* (हिमवृष्टी) यांसारखे शब्द *हवामान अंदाज* दर्शवण्याची शक्यता असते, तर *stocks* (शेअर्स), *dollar* (डॉलर) यांसारखे शब्द *आर्थिक बातम्या* याकडे झुकतात.

**शब्दांची पिशवी** (BoW) वेक्टर प्रतिनिधित्व हे पारंपरिक वेक्टर प्रतिनिधित्वांपैकी सर्वात जास्त वापरले जाणारे आहे. प्रत्येक शब्द एका वेक्टर निर्देशांकाशी जोडलेला असतो, आणि वेक्टर घटक दिलेल्या दस्तऐवजात त्या शब्दाच्या उपस्थितीची संख्या दर्शवतो.

![शब्दांची पिशवी वेक्टर प्रतिनिधित्व स्मृतीत कसे सादर केले जाते हे दाखवणारी प्रतिमा.](../../../../../translated_images/mr/bag-of-words-example.606fc1738f1d7ba9.webp)

> **टीप**: तुम्ही BoW ला मजकूरातील स्वतंत्र शब्दांसाठी असलेल्या सर्व एक-हॉट-एन्कोडेड वेक्टरच्या बेरीजप्रमाणे देखील विचार करू शकता.

खाली Scikit Learn पायथन लायब्ररी वापरून शब्दांची पिशवी प्रतिनिधित्व कसे तयार करायचे याचे एक उदाहरण दिले आहे:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

आमच्या AG_NEWS डेटासेटच्या व्हेक्टर प्रतिनिधीतून बॅग-ऑफ-वर्ड्स व्हेक्टर गणना करण्यासाठी, आम्ही खालील फंक्शन वापरू शकतो:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **टीप:** येथे आम्ही जागतिक `vocab_size` व्हेरिएबल वापरत आहोत शब्दसंग्रहाचा डीफॉल्ट आकार निर्दिष्ट करण्यासाठी. कारण अनेकदा शब्दसंग्रहाचा आकार खूप मोठा असतो, आपण सर्वात वारंवार वापरल्या जाणाऱ्या शब्दांपर्यंत शब्दसंग्रहाचा आकार मर्यादित करू शकतो. `vocab_size` मूल्य कमी करण्याचा प्रयत्न करा आणि खालील कोड चालवा, आणि त्याचा अचूकतेवर कसा परिणाम होतो ते पहा. तुम्हाला काही प्रमाणात अचूकतेत घट अपेक्षित असावी, परंतु ती नाट्यमय नसावी, उच्च कार्यक्षमतेच्या बदल्यात.


## BoW वर्गीकरणकर्ता प्रशिक्षण

आता आपण आपल्या मजकुराचे बॅग-ऑफ-वर्ड्स (BoW) प्रतिनिधित्व कसे तयार करायचे ते शिकलो आहोत, चला त्यावर आधारित एक वर्गीकरणकर्ता प्रशिक्षण देऊया. सर्वप्रथम, आपल्याला आपल्या डेटासेटचे प्रशिक्षणासाठी असे रूपांतर करणे आवश्यक आहे की, सर्व स्थिती वेक्टर प्रतिनिधित्व बॅग-ऑफ-वर्ड्स प्रतिनिधित्वामध्ये रूपांतरित होतील. हे `bowify` फंक्शनला `collate_fn` पॅरामीटर म्हणून स्टँडर्ड torch `DataLoader` मध्ये पास करून साध्य केले जाऊ शकते:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

आता आपण एक साधा वर्गीकरण करणारा न्यूरल नेटवर्क परिभाषित करूया ज्यामध्ये एकच रेखीय स्तर आहे. इनपुट व्हेक्टरचा आकार `vocab_size` च्या बरोबरीचा आहे, आणि आउटपुटचा आकार वर्गांच्या संख्येशी (4) संबंधित आहे. कारण आपण वर्गीकरण कार्य सोडवत आहोत, अंतिम सक्रियता फंक्शन `LogSoftmax()` आहे.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

आता आपण मानक PyTorch प्रशिक्षण लूप परिभाषित करू. कारण आपला डेटासेट खूप मोठा आहे, शिक्षणाच्या उद्देशाने आपण फक्त एका epoch साठी प्रशिक्षण देऊ, आणि कधी कधी एका epoch पेक्षा कमी (प्रशिक्षण मर्यादित करण्यासाठी `epoch_size` पॅरामीटर निर्दिष्ट करण्याची परवानगी देते). प्रशिक्षणादरम्यान संचित प्रशिक्षण अचूकता देखील आम्ही नोंदवू; नोंदवण्याची वारंवारता `report_freq` पॅरामीटर वापरून निर्दिष्ट केली जाते.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## बायग्राम्स, ट्रायग्राम्स आणि एन-ग्राम्स

बॅग ऑफ वर्ड्स पद्धतीची एक मर्यादा म्हणजे काही शब्द हे बहु-शब्द अभिव्यक्तींचा भाग असतात. उदाहरणार्थ, 'hot dog' या शब्दाचा अर्थ इतर संदर्भांतील 'hot' आणि 'dog' या शब्दांपेक्षा पूर्णपणे वेगळा असतो. जर आपण 'hot' आणि 'dog' या शब्दांचे नेहमी समान व्हेक्टरद्वारे प्रतिनिधित्व केले, तर ते आपल्या मॉडेलसाठी गोंधळ निर्माण करू शकते.

यावर उपाय म्हणून, **एन-ग्राम प्रतिनिधित्व** दस्तऐवज वर्गीकरणाच्या पद्धतींमध्ये वापरले जाते, जिथे प्रत्येक शब्द, द्वि-शब्द किंवा त्रि-शब्दाची वारंवारता वर्गीकरण करणारे मॉडेल प्रशिक्षणासाठी उपयुक्त वैशिष्ट्य ठरते. उदाहरणार्थ, बायग्राम प्रतिनिधित्वामध्ये, मूळ शब्दांव्यतिरिक्त, सर्व शब्दजोड्या शब्दसंग्रहात समाविष्ट केल्या जातात.

खाली दिलेले उदाहरण स्कायकीट लर्न वापरून बायग्राम बॅग ऑफ वर्ड्स प्रतिनिधित्व कसे तयार करायचे हे दर्शवते:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

N-gram पद्धतीचा मुख्य तोटा म्हणजे शब्दसंग्रहाचा आकार खूप वेगाने वाढतो. प्रत्यक्षात, आपल्याला N-gram चे प्रतिनिधित्व काही परिमाण कमी करण्याच्या तंत्रांसह एकत्र करावे लागते, जसे की *embeddings*, ज्याबद्दल आपण पुढील युनिटमध्ये चर्चा करू.

**AG News** डेटासेटमध्ये N-gram चे प्रतिनिधित्व वापरण्यासाठी, आपल्याला विशेष ngram शब्दसंग्रह तयार करावा लागेल:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


आपण वर दिलेल्या कोडचा वापर करून वर्गीकरणकर्ता प्रशिक्षित करू शकतो, परंतु हे खूप मेमरी-अकार्यक्षम ठरेल. पुढील युनिटमध्ये, आपण एम्बेडिंग्स वापरून बिग्राम वर्गीकरणकर्ता प्रशिक्षित करू.

> **टीप:** तुम्ही फक्त ते ngrams ठेवू शकता जे मजकुरात निर्दिष्ट केलेल्या वेळेपेक्षा जास्त वेळा येतात. यामुळे कमी वारंवार येणारे बिग्राम वगळले जातील आणि परिमाणात्मकता लक्षणीयरीत्या कमी होईल. हे करण्यासाठी, `min_freq` पॅरामीटरला उच्च मूल्यावर सेट करा आणि शब्दसंग्रहाच्या लांबीतील बदल निरीक्षण करा.


## टर्म फ्रिक्वेंसी इन्व्हर्स डॉक्युमेंट फ्रिक्वेंसी TF-IDF

BoW (बॅग ऑफ वर्ड्स) प्रतिनिधित्वामध्ये, शब्दांच्या उपस्थितीला समान वजन दिले जाते, शब्द स्वतः कोणताही विचार न करता. परंतु, हे स्पष्ट आहे की वारंवार येणारे शब्द, जसे की *a*, *in*, इत्यादी वर्गीकरणासाठी कमी महत्त्वाचे असतात, विशेष शब्दांच्या तुलनेत. खरं तर, बहुतेक NLP कार्यांमध्ये काही शब्द इतरांपेक्षा अधिक महत्त्वाचे असतात.

**TF-IDF** म्हणजे **टर्म फ्रिक्वेंसी–इन्व्हर्स डॉक्युमेंट फ्रिक्वेंसी**. हे बॅग ऑफ वर्ड्सचे एक प्रकार आहे, ज्यामध्ये 0/1 द्विआधारी मूल्याच्या ऐवजी, जे एखाद्या दस्तऐवजात शब्दाच्या उपस्थितीचे सूचक असते, एक फ्लोटिंग-पॉइंट मूल्य वापरले जाते, जे कॉर्पसमध्ये शब्दाच्या उपस्थितीच्या वारंवारतेशी संबंधित असते.

अधिक औपचारिकपणे, दस्तऐवज $j$ मधील शब्द $i$ चे वजन $w_{ij}$ खालीलप्रमाणे परिभाषित केले जाते:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
जिथे
* $tf_{ij}$ म्हणजे $j$ मध्ये $i$ च्या उपस्थितीची संख्या, म्हणजेच आपण यापूर्वी पाहिलेले BoW मूल्य
* $N$ म्हणजे संग्रहातील दस्तऐवजांची संख्या
* $df_i$ म्हणजे संपूर्ण संग्रहात शब्द $i$ असलेल्या दस्तऐवजांची संख्या

TF-IDF मूल्य $w_{ij}$ दस्तऐवजामध्ये एखादा शब्द किती वेळा दिसतो याच्या प्रमाणात वाढते आणि कॉर्पसमध्ये त्या शब्दाचा समावेश असलेल्या दस्तऐवजांच्या संख्येने समायोजित केले जाते, ज्यामुळे काही शब्द इतरांपेक्षा अधिक वारंवार दिसतात याचा विचार केला जातो. उदाहरणार्थ, जर एखादा शब्द संग्रहातील *प्रत्येक* दस्तऐवजामध्ये दिसतो, तर $df_i=N$, आणि $w_{ij}=0$, आणि असे शब्द पूर्णपणे दुर्लक्षित केले जातील.

तुम्ही Scikit Learn वापरून सहजपणे टेक्स्टचे TF-IDF व्हेक्टरायझेशन तयार करू शकता:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## निष्कर्ष

जरी TF-IDF प्रतिनिधित्व वेगवेगळ्या शब्दांना वारंवारतेचे वजन प्रदान करत असले तरी ते अर्थ किंवा क्रम दर्शवू शकत नाहीत. प्रसिद्ध भाषाशास्त्रज्ञ जे. आर. फर्थ यांनी 1935 मध्ये म्हटल्याप्रमाणे, “शब्दाचा संपूर्ण अर्थ नेहमीच संदर्भात्मक असतो, आणि संदर्भाशिवाय अर्थाचा अभ्यास गंभीरपणे घेतला जाऊ शकत नाही.”. या अभ्यासक्रमात पुढे आपण भाषेचे मॉडेलिंग वापरून मजकुरातून संदर्भात्मक माहिती कशी मिळवायची ते शिकू.



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात ठेवा की स्वयंचलित भाषांतरे त्रुटी किंवा अचूकतेच्या अभावाने युक्त असू शकतात. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी, व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून उद्भवलेल्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
